<a href="https://colab.research.google.com/github/zakariahasan/Deep-Learning/blob/master/colab_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# Importing the dataset
dataset = pd.read_csv('/content/drive/My Drive/ML&DL/Data/Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13].values


In [4]:
#Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)


In [5]:
## Concatenate the Data Frames
X=pd.concat([X,geography,gender],axis=1)


In [6]:
## Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

In [7]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [8]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
## Perform Hyperparameter Optimization with Scikit_learn Wrapper

In [10]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from tensorflow.keras.activations import relu, sigmoid

In [11]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) 
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [12]:
model = KerasClassifier(build_fn=create_model, verbose=0)

In [13]:
%%time
layers = ([20],(21,40), (45, 30, 15))
activations = ('sigmoid', 'relu')
param_grid = dict(layers=layers, activation=activations, batch_size = (128, 256), epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

grid_result = grid.fit(X_train, y_train)

[grid_result.best_score_,grid_result.best_params_]


CPU times: user 2min 26s, sys: 10.9 s, total: 2min 37s
Wall time: 2min 16s


In [14]:
grid_result

GridSearchCV(cv=5, error_score=nan,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f2bebf4d9b0>,
             iid='deprecated', n_jobs=None,
             param_grid={'activation': ('sigmoid', 'relu'),
                         'batch_size': (128, 256), 'epochs': [30],
                         'layers': ([20], (21, 40), (45, 30, 15))},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [16]:
#!pip install -q -U keras-tuner
import kerastuner as kt

In [17]:
## Perform Hyperparameter Optimization with Kerastuner

In [18]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch


def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu',input_dim=X_train.shape[1]))
    model.add(layers.Dense(1, activation='sigmoid',kernel_initializer= 'glorot_uniform'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [19]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='test_pro')

In [20]:
tuner.search_space_summary()

In [21]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Epoch 1/5
250/250 [==============================] - 1s 3ms/step - loss: 0.4704 - accuracy: 0.7870 - val_loss: 0.4189 - val_accuracy: 0.8160
Epoch 2/5
250/250 [==============================] - 1s 2ms/step - loss: 0.4084 - accuracy: 0.8273 - val_loss: 0.3917 - val_accuracy: 0.8355
Epoch 3/5
250/250 [==============================] - 1s 3ms/step - loss: 0.3856 - accuracy: 0.8386 - val_loss: 0.3692 - val_accuracy: 0.8505
Epoch 4/5
250/250 [==============================] - 1s 3ms/step - loss: 0.3682 - accuracy: 0.8486 - val_loss: 0.3576 - val_accuracy: 0.8550
Epoch 5/5
250/250 [==============================] - 1s 2ms/step - loss: 0.3583 - accuracy: 0.8530 - val_loss: 0.3505 - val_accuracy: 0.8580
Epoch 1/5
250/250 [==============================] - 1s 3ms/step - loss: 0.4755 - accuracy: 0.7903 - val_loss: 0.4183 - val_accuracy: 0.8265
Epoch 2/5
250/250 [==============================] - 1s 3ms/step - loss: 0.4101 - accuracy: 0.8250 - val_loss: 0.3945 - val_accuracy: 0.8460
Epoch 3/5
250

Epoch 1/5
250/250 [==============================] - 1s 3ms/step - loss: 0.3929 - accuracy: 0.8400 - val_loss: 0.3567 - val_accuracy: 0.8510
Epoch 2/5
250/250 [==============================] - 1s 3ms/step - loss: 0.3590 - accuracy: 0.8508 - val_loss: 0.3652 - val_accuracy: 0.8535
Epoch 3/5
250/250 [==============================] - 1s 3ms/step - loss: 0.3547 - accuracy: 0.8572 - val_loss: 0.3410 - val_accuracy: 0.8625
Epoch 4/5
250/250 [==============================] - 1s 3ms/step - loss: 0.3493 - accuracy: 0.8577 - val_loss: 0.3390 - val_accuracy: 0.8695
Epoch 5/5
250/250 [==============================] - 1s 3ms/step - loss: 0.3448 - accuracy: 0.8595 - val_loss: 0.3457 - val_accuracy: 0.8590
Epoch 1/5
250/250 [==============================] - 1s 3ms/step - loss: 0.3909 - accuracy: 0.8420 - val_loss: 0.3521 - val_accuracy: 0.8575
Epoch 2/5
250/250 [==============================] - 1s 2ms/step - loss: 0.3609 - accuracy: 0.8512 - val_loss: 0.3464 - val_accuracy: 0.8635
Epoch 3/5
250

Epoch 1/5
250/250 [==============================] - 1s 3ms/step - loss: 0.5844 - accuracy: 0.7234 - val_loss: 0.4894 - val_accuracy: 0.8025
Epoch 2/5
250/250 [==============================] - 1s 2ms/step - loss: 0.4627 - accuracy: 0.8026 - val_loss: 0.4404 - val_accuracy: 0.8110
Epoch 3/5
250/250 [==============================] - 1s 2ms/step - loss: 0.4340 - accuracy: 0.8119 - val_loss: 0.4227 - val_accuracy: 0.8195
Epoch 4/5
250/250 [==============================] - 1s 3ms/step - loss: 0.4209 - accuracy: 0.8175 - val_loss: 0.4124 - val_accuracy: 0.8260
Epoch 5/5
250/250 [==============================] - 1s 3ms/step - loss: 0.4124 - accuracy: 0.8226 - val_loss: 0.4042 - val_accuracy: 0.8275
Epoch 1/5
250/250 [==============================] - 1s 3ms/step - loss: 0.5669 - accuracy: 0.7552 - val_loss: 0.4867 - val_accuracy: 0.7975
Epoch 2/5
250/250 [==============================] - 1s 2ms/step - loss: 0.4619 - accuracy: 0.7985 - val_loss: 0.4422 - val_accuracy: 0.8065
Epoch 3/5
250

Epoch 1/5
250/250 [==============================] - 1s 3ms/step - loss: 0.4331 - accuracy: 0.8105 - val_loss: 0.3841 - val_accuracy: 0.8455
Epoch 2/5
250/250 [==============================] - 1s 3ms/step - loss: 0.3742 - accuracy: 0.8468 - val_loss: 0.3559 - val_accuracy: 0.8555
Epoch 3/5
250/250 [==============================] - 1s 3ms/step - loss: 0.3552 - accuracy: 0.8545 - val_loss: 0.3469 - val_accuracy: 0.8640
Epoch 4/5
250/250 [==============================] - 1s 3ms/step - loss: 0.3471 - accuracy: 0.8596 - val_loss: 0.3464 - val_accuracy: 0.8545
Epoch 5/5
250/250 [==============================] - 1s 3ms/step - loss: 0.3449 - accuracy: 0.8591 - val_loss: 0.3420 - val_accuracy: 0.8590
Epoch 1/5
250/250 [==============================] - 1s 3ms/step - loss: 0.4449 - accuracy: 0.7981 - val_loss: 0.3806 - val_accuracy: 0.8490
Epoch 2/5
250/250 [==============================] - 1s 2ms/step - loss: 0.3729 - accuracy: 0.8456 - val_loss: 0.3505 - val_accuracy: 0.8580
Epoch 3/5
250

Epoch 1/5
250/250 [==============================] - 1s 3ms/step - loss: 0.5853 - accuracy: 0.7398 - val_loss: 0.5027 - val_accuracy: 0.7975
Epoch 2/5
250/250 [==============================] - 1s 3ms/step - loss: 0.4749 - accuracy: 0.7970 - val_loss: 0.4518 - val_accuracy: 0.8010
Epoch 3/5
250/250 [==============================] - 1s 3ms/step - loss: 0.4439 - accuracy: 0.8031 - val_loss: 0.4308 - val_accuracy: 0.8100
Epoch 4/5
250/250 [==============================] - 1s 3ms/step - loss: 0.4294 - accuracy: 0.8119 - val_loss: 0.4198 - val_accuracy: 0.8180
Epoch 5/5
250/250 [==============================] - 1s 3ms/step - loss: 0.4204 - accuracy: 0.8163 - val_loss: 0.4119 - val_accuracy: 0.8250
Epoch 1/5
250/250 [==============================] - 1s 3ms/step - loss: 0.5561 - accuracy: 0.7906 - val_loss: 0.4913 - val_accuracy: 0.7975
Epoch 2/5
250/250 [==============================] - 1s 2ms/step - loss: 0.4680 - accuracy: 0.7974 - val_loss: 0.4471 - val_accuracy: 0.8045
Epoch 3/5
250

INFO:tensorflow:Oracle triggered exit


In [24]:
models = tuner.get_best_models(num_models=2)
models

In [23]:
tuner.results_summary()